<a href="https://colab.research.google.com/github/epb378/Datamining_for_ligands/blob/main/defining%20functions%20to%20take%20article%20to%20sdfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1: Scrape HTML document for chemicals**

In [ ]:

#trial data to test pipeline
!wget -O trialhtml.html https://pubs.rsc.org/en/content/articlehtml/2009/jm/b901933b

--2020-11-12 16:03:40--  https://pubs.rsc.org/en/content/articlehtml/2009/jm/b901933b
Resolving pubs.rsc.org (pubs.rsc.org)... 78.25.196.229
Connecting to pubs.rsc.org (pubs.rsc.org)|78.25.196.229|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /en/content/articlelanding/2009/jm/b901933b/unauth [following]
--2020-11-12 16:03:41--  https://pubs.rsc.org/en/content/articlelanding/2009/jm/b901933b/unauth
Reusing existing connection to pubs.rsc.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 141997 (139K) [text/html]
Saving to: ‘trialhtml.html’

trialhtml.html      100%[===================>] 138.67K   502KB/s    in 0.3s    

2020-11-12 16:03:41 (502 KB/s) - ‘trialhtml.html’ saved [141997/141997]



In [ ]:
#get chemicalfilters.py and ligandfinder.py from github
!wget https://raw.githubusercontent.com/epb378/Datamining_for_ligands/main/chemicalfilters.py
!wget https://raw.githubusercontent.com/epb378/Datamining_for_ligands/main/ligandfinder.py


--2020-11-12 15:56:49--  https://raw.githubusercontent.com/epb378/Datamining_for_ligands/main/chemicalfilters.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 292 [text/plain]
Saving to: ‘chemicalfilters.py.2’

chemicalfilters.py. 100%[===================>]     292  --.-KB/s    in 0s      

2020-11-12 15:56:49 (17.8 MB/s) - ‘chemicalfilters.py.2’ saved [292/292]

--2020-11-12 15:56:49--  https://raw.githubusercontent.com/epb378/Datamining_for_ligands/main/ligandfinder.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1119 (1.1K) [text/pl

In [ ]:
!pip install chemdataextractor
!pip install mendeleev
!pip install kora -q
!cde data download
!pip install json2html


Successfully downloaded 0 new data packages (18 existing)
  Created wheel for json2html: filename=json2html-1.3.0-cp36-none-any.whl size=7595 sha256=c8cfbc43672198c723aca16d6a67c6cfa36e1e997d8063d24ed7b628d189e795
  Stored in directory: /root/.cache/pip/wheels/58/36/ad/386be30507bcb8f0c9830004bd776132eba63c1b945ed79255
Successfully built json2html


In [ ]:
from tqdm import tqdm
import time
import urllib.request
import json
import pandas as pd
import kora.install.rdkit
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
import subprocess
import requests, zipfile, io
cids_url_head='https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/'
cids_url_tail='/cids/json?name_type=word' #THIS IS FOR PARTIAL MATCHES_returns a list of CIDS
#info about this here https://pubchemdocs.ncbi.nlm.nih.gov/pug-rest-tutorial
SMILES_url_head='https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/'
SMILES_url_tail='/property/CanonicalSMILES/txt'
wait_time=0.5
r = requests.get( 'https://git.durrantlab.pitt.edu/jdurrant/dimorphite_dl/-/archive/1.2.4/dimorphite_dl-1.2.4.zip' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
import chemicalfilters as cf
import ligandfinder as lf
import json2html


def dimophite_run(smile, min_ph, max_ph):
  process = subprocess.run(['python','dimorphite_dl-1.2.4/dimorphite_dl.py', '--smiles', smile, '--min_ph', str(min_ph), '--max_ph', str(max_ph), '--silent'], check=True, stdout=subprocess.PIPE, universal_newlines=True)
  output = process.stdout
  output_list=output.split(sep='\t\n')
  output_list = list(filter(None, output_list))
  return output_list

def gen3Dsdf(mol):
  m3 = Chem.AddHs(mol)
  AllChem.EmbedMolecule(m3,randomSeed=0xf00d)
  return Chem.MolToMolBlock(m3)


def chem_names_to_smiles(chems_list):
    ##chemical name to cids to smiles
  for i in range(0,len(chems_list)):
    chem_url=cids_url_head+chems_list[i][0]+cids_url_tail
    try:
      with urllib.request.urlopen(chem_url) as response:
        data=json.load(response)
      cids_list.append(data)
      bare_cid=cids_list[i]['IdentifierList']['CID'][0]
      SMILES_url=SMILES_url_head+str(bare_cid)+SMILES_url_tail
      time.sleep(wait_time)
      try:
        with urllib.request.urlopen(SMILES_url) as response:
          data=response.read().decode("utf-8")
        SMILES_list.append(data)
      except:
        SMILES_list.append('')
      time.sleep(wait_time)
    except:
      time.sleep(wait_time)
      cids_list.append('')

    return SMILES_list

In [ ]:
test_article= 'https://pubs.rsc.org/en/content/articlehtml/2012/cs/c2cs35197h'

chems_list=['acetone','toluene','acetic acid','hexanoic acid','potassium nitrate']
cids_list=[]
SMILES_list=[]

In [ ]:
chems_list=lf.ligandfinder('trialhtml.html',0)
print(chems_list)

[('Tel', 0), ('TiO2', 0), ('Al2O3', 0), ('ZrO2', 0), ('MnO', 0), ('CeO2', 0), ('Anupreet Kaur', 0)]


**Step 2: Convert scraped chemicals to cids, smiles, enumerated smiles and finally SDFs**

In [ ]:
##chemical name to cids to smiles
for i in tqdm(range(0,len(chems_list))):
  chem_url=cids_url_head+chems_list[i][0]+cids_url_tail
  try:
    with urllib.request.urlopen(chem_url) as response:
      data=json.load(response)
    cids_list.append(data)
    bare_cid=cids_list[i]['IdentifierList']['CID'][0]
    SMILES_url=SMILES_url_head+str(bare_cid)+SMILES_url_tail
    time.sleep(wait_time)
    try:
      with urllib.request.urlopen(SMILES_url) as response:
        data=response.read().decode("utf-8")
      SMILES_list.append(data)
    except:
      SMILES_list.append('')
    time.sleep(wait_time)
  except:
    time.sleep(wait_time)
    cids_list.append('')
  

100%|██████████| 7/7 [00:08<00:00,  1.26s/it]


['CC[Pb](CC)(CC)CC\n', 'O=[Ti]=O\n', 'O=[Al]O[Al]=O\n', 'O=[Zr]=O\n', 'C1=CC2=C(C=C3C4=C2C(=C1)C(=O)C5=CC(=C6C=CC=C(C6=C54)C3=O)Br)Br\n', 'O=[Ce]=O\n']


**Step 2.2:** Enumerate charges

In [ ]:
extra_smiles=[]
for smile in SMILES_list:
  extra_smiles= extra_smiles+dimophite_run(smile, 4.4, 8.4)
SMILES_list=extra_smiles

In [ ]:
mols = [Chem.MolFromSmiles(smi) for smi in SMILES_list]

**Step 2.3:** All smiles to sdfs

In [ ]:
mols_sdfs=[]
for x in tqdm(mols):
  try: 
    mols_sdfs.append(gen3Dsdf(x))
  except:
    pass

100%|██████████| 12/12 [00:00<00:00, 165.09it/s]


In [ ]:
print(SMILES_list[1] + '\n' + mols_sdfs[1] +'\n' + '$$$$ \n' )

O=[Ti]=O

     RDKit          3D

  3  2  0  0  0  0  0  0  0  0999 V2000
   -1.6344   -0.4392   -0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
   -0.4242    0.6534   -0.0000 Ti  0  0  0  0  0  4  0  0  0  0  0  0
    2.0587   -0.2141    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  2  0
  2  3  2  0
M  END

$$$$ 



In [ ]:
### writing sdfs to file

with open('your_file.txt', 'w') as f:
    for i in range(0,len(SMILES_list)):
        f.write(SMILES_list[i] + '\n' + mols_sdfs[i] +'\n' + '$$$$ \n') ###this format will need to be changed to make it readable by the docking software

In [ ]:
def article_to_sdfs(html_file, output_file=None, index=0, min_pH=4.4, max_pH=8.4):
  if output_file==None:
    output_file='temp_output.txt'
  chems_list=lf.ligandfinder(html_file,index) #scrape HTML for ligands/surfactant/coordinating solvents etc.
  SMILES_list=chem_names_to_smiles(chems_list) #use pubchem to search for SMILES from chem names (via CIDS)
  extra_smiles=[]
  for smile in SMILES_list:
    extra_smiles= extra_smiles+dimophite_run(smile, min_pH, max_pH) #See if there are any other charged states in the pH range of interest
  SMILES_list=extra_smiles
  mols = [Chem.MolFromSmiles(smi) for smi in SMILES_list] # use RDkit to convert smiles to mol format
  mols_sdfs=[]
  for x in mols:
    try: 
      mols_sdfs.append(gen3Dsdf(x)) #try to turn these mols formats into SDF docs
    except:
      pass  
  with open(output_file, 'w') as f:
    for i in range(0,len(SMILES_list)):
        f.write(SMILES_list[i] + '\n' + mols_sdfs[i] +'\n' + '$$$$ \n') ###this format will need to be changed to make it readable by the docking software


In [ ]:
article_to_sdfs('trialhtml.html')